In [24]:
pip install wandb


  Using cached wandb-0.14.0-py3-none-any.whl (2.0 MB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)
  Using cached pathtools-0.1.2.tar.gz (11 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached sentry_sdk-1.18.0-py2.py3-none-any.whl (194 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8792 sha256=6136a9864c84cda9edb0a6caba801499c4319e0109ad209622b1d5df36d3d4b2
  Stored in directory: c:\users\letai\appdata\local\pip\cache\wheels\20\7c\09\4ad42725a29fce4bc21137c7f25f062b3655a4aea5b0e8d9a2
Successfully built pathtools


In [25]:
import numpy as np
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision.datasets import OxfordIIITPet
import torch
import matplotlib.pyplot as plt
import datasets, models, train,utils

In [26]:
def load_dataset(split="trainval", root="/tmp/adl_data") -> Dataset:
    """
    returns a Dataset with:
    - images as normalized float32 tensors
    - labels as uint 8 tensors

    the labels are:
    - 1: foreground
    - 2: background
    - 3: not classified
    """
    image_transform = torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )
    label_transform = lambda label: torch.from_numpy(np.array(label))
    ds = OxfordIIITPet(
        root=root,
        split=split,
        target_types="segmentation",
        download=True,
        transform=image_transform,
        target_transform=label_transform,
    )
    return ds

In [27]:
def create_datasets(data_dir, valid_frac, labelled_frac):
    trainval_ds = load_dataset('trainval')
    test_ds = load_dataset('test')

    rng = torch.Generator()
    rng.manual_seed(0)
    train_all_ds, valid_ds = torch.utils.data.random_split(trainval_ds, (1-valid_frac, valid_frac), generator=rng)
    _, train_lab_ds = torch.utils.data.random_split(train_all_ds, (1-labelled_frac, labelled_frac), generator=rng)
    
    return train_all_ds, train_lab_ds, valid_ds

In [28]:
data_dir = 'data'
valid_frac = 0.2
labelled_frac = 1

In [29]:
train_all_ds, train_lab_ds, valid_ds = create_datasets(data_dir,valid_frac,labelled_frac)

c:\Users\letai\anaconda3\lib\site-packages\torch\utils\data\dataset.py:342: UserWarning: Length of split at index 0 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "


In [30]:
import torch
from torchvision.models import resnet50, resnet101, ResNet50_Weights, ResNet101_Weights
from torchvision.models.segmentation import deeplabv3_resnet50, deeplabv3_resnet101


def load_deeplab(num_classes=1, use_imagenet_weights=True, large_resnet=False):
    """
    Initializes a DeepLabV3 segmentation model according to the paper
    'Semi-Supervised Semantic Segmentation with Cross Pseudo Supervision'

    The model outputs a dictionary where the key 'out' contains
    the predicted mask. If the input images have shape [N, C, H, W], the
    predicted mask has shape [N, 1, H, W]. The '1' corresponds to the
    fact that we are only outputting a single logit, corresponding to
    foreground/background:
    >>> logits = model(images)['out'][:, 0]

    And to get probabilities from the logits, run:
    >>> probs = torch.sigmoid(logits)

    Known bug: the model requires at least two images in a batch.
    """

    # load deeplab with random initialization

    deeplab = (deeplabv3_resnet101 if large_resnet else deeplabv3_resnet50)(
        num_classes=num_classes
    )

    # optionally initialize the backbone with imagenet weights
    if use_imagenet_weights:
        # load a resnet50 pretrained on imagenet
        weights = (
            ResNet101_Weights.IMAGENET1K_V2
            if large_resnet
            else ResNet50_Weights.IMAGENET1K_V2
        )
        resnet = (resnet101 if large_resnet else resnet50)(weights=weights)

        # transfer the resnet's weights to the deeplab's backbone
        deeplab.backbone.load_state_dict(resnet.state_dict(), strict=False)

        # check that each layer in deeplab.backbone has the same weights as the resnet
        for name in deeplab.backbone.state_dict().keys():
            assert torch.allclose(
                resnet.state_dict()[name], deeplab.backbone.state_dict()[name]
            )

    return deeplab

In [31]:
load_deeplab()

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [32]:
print(len(valid_ds))

736


In [33]:
len(test_ds), len(valid_ds), len(unlab_train_ds), len(lab_train_ds)

NameError: name 'test_ds' is not defined